# Model.fit() input as ```tf.data.Dataset```

* [tf.keras.Model.fit()](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit)

> A tf.data dataset. Should return a tuple of either **(inputs, targets)** or (inputs, targets, sample_weights).

* [How do I train my keras model using Tf Datasets #720](https://github.com/tensorflow/datasets/issues/720)

> You can use ```tfds.load(as_supervised=True)``` kwargs to return an **```(image, label)``` tuple expected by keras**. 
For images, you would have in addition to cast/normalize the image to tf.float32, for this, you can use ```tf.data.Dataset.map```.
> 
> ```
> def _normalize_img(img, label):
>   img = tf.cast(img, tf.float32) / 255.
>   return (img, label)
> 
> ds = tfds.load('mnist', split='train', as_supervised=True)
> ds = ds.batch(32)
> ds = ds.map(_normalize_img)
>
> model.fit(ds_train, epochs=5)
> ```
 
* [How does tf.keras.Model tell between features and label(s) in tf.data.Dataset and in TFRecords?](https://stackoverflow.com/a/59838140/4281353) 

> As such, the dataset that is given to model.fit is actually **a dataset of tuples**, and to the best of my knowledge, this is exactly what the model will assume if you provide a tf.data.Dataset as input to the fit function -- **a dataset of tuples (inputs, labels)**. So the first will be taken as input to the model, the second as target for the loss function.

* [Support model.fit using targets in a dictionary](https://github.com/tensorflow/tensorflow/issues/24962#issuecomment-475709720)

> ```
> def make_dataset(images, labels, batch_size=64, buffer_size=1024, shuffle=True):
>     inputs = dict(images=images)
>     outputs = dict(labels=labels)
>     dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))
>     if shuffle:
>         dataset = dataset.shuffle(buffer_size=buffer_size)
>     dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
>     dataset = dataset.map(map_func=normalize_fn, num_parallel_calls=8)
>     dataset = dataset.batch(batch_size)
>     return dataset
> 
> model.add(tf.keras.Input(shape=(28, 28, 1), name='images'))
> model.add(tf.keras.layers.Dense(10, activation='softmax', name='labels'))
> ```

# Baics

1. Generate a ```tf.data.Dataset``` that returns ```(data, label)```.
2. Use tf.data.Dataset.from_tensor_slices((inputs, outputs)) where inputs and outputs are separate sequences.

## Batch Shape

```
model.fit(dataset)
---
ValueError: Input 0 of layer "model_6" is incompatible with the layer:
expected shape=(None, 448, 448, 3), found shape=(448, 448, 3)
```

```tf.keras.Model.fit()``` expects batches. DO not forget ```tf.data.Dataset.batch(batch_size)``` to have the batched shape.
```
model.fit(dataset.batch(batch_size))
```